In [ ]:
import pandas as pd

# Cargar los archivos de las tres etapas
ruta_etapa1 = 'Resultados_Etapa1.xlsx'
ruta_etapa2 = 'Resultados_Etapa2.xlsx'
ruta_etapa3 = 'Resultados_Etapa3.xlsx'

# Cargar las hojas relevantes de cada etapa
etapa1_mejores = pd.read_excel(ruta_etapa1, sheet_name='Mejores Modelos')
etapa1_diferencias = pd.read_excel(ruta_etapa1, sheet_name='Diferencias %')

etapa2_mejores = pd.read_excel(ruta_etapa2, sheet_name='Mejores Modelos')
etapa2_diferencias = pd.read_excel(ruta_etapa2, sheet_name='Diferencias %')

etapa3_mejores = pd.read_excel(ruta_etapa3, sheet_name='Mejores Modelos')
etapa3_diferencias = pd.read_excel(ruta_etapa3, sheet_name='Diferencias %')

# Ajustar los DataFrames para eliminar la columna 'Unnamed: 0' y usar los nombres correctos de índices
etapa1_mejores = etapa1_mejores.set_index(etapa1_mejores.columns[0])
etapa2_mejores = etapa2_mejores.set_index(etapa2_mejores.columns[0])
etapa3_mejores = etapa3_mejores.set_index(etapa3_mejores.columns[0])

# Unificar las hojas de diferencias para compararlas
diferencias_combined = etapa1_diferencias.add_prefix('Etapa 1 - ')
diferencias_combined = diferencias_combined.join(etapa2_diferencias.add_prefix('Etapa 2 - '), how='outer')
diferencias_combined = diferencias_combined.join(etapa3_diferencias.add_prefix('Etapa 3 - '), how='outer')

# Crear los DataFrames finales
final_mejores_modelos_df = pd.DataFrame(index=diferencias_combined.index, columns=etapa1_mejores.columns)
final_diferencias_df = pd.DataFrame(index=diferencias_combined.index, columns=etapa1_mejores.columns)

# Iterar sobre cada subproceso y mes para determinar el mejor modelo general
for subproceso in diferencias_combined.index:
    for mes in etapa1_mejores.columns:
        # Recoger los valores de diferencias y nombres de modelos de cada etapa
        diferencias_etapa1 = diferencias_combined.loc[subproceso, f'Etapa 1 - {mes}'] if f'Etapa 1 - {mes}' in diferencias_combined.columns else float('inf')
        diferencias_etapa2 = diferencias_combined.loc[subproceso, f'Etapa 2 - {mes}'] if f'Etapa 2 - {mes}' in diferencias_combined.columns else float('inf')
        diferencias_etapa3 = diferencias_combined.loc[subproceso, f'Etapa 3 - {mes}'] if f'Etapa 3 - {mes}' in diferencias_combined.columns else float('inf')
        
        modelo_etapa1 = etapa1_mejores.loc[subproceso, mes] if mes in etapa1_mejores.columns else None
        modelo_etapa2 = etapa2_mejores.loc[subproceso, mes] if mes in etapa2_mejores.columns else None
        modelo_etapa3 = etapa3_mejores.loc[subproceso, mes] if mes in etapa3_mejores.columns else None

        # Determinar cuál de las diferencias es la menor y seleccionar el modelo correspondiente
        min_diff = min(diferencias_etapa1, diferencias_etapa2, diferencias_etapa3)
        
        if min_diff == diferencias_etapa1:
            final_mejores_modelos_df.loc[subproceso, mes] = modelo_etapa1
            final_diferencias_df.loc[subproceso, mes] = diferencias_etapa1
        elif min_diff == diferencias_etapa2:
            final_mejores_modelos_df.loc[subproceso, mes] = modelo_etapa2
            final_diferencias_df.loc[subproceso, mes] = diferencias_etapa2
        else:
            final_mejores_modelos_df.loc[subproceso, mes] = modelo_etapa3
            final_diferencias_df.loc[subproceso, mes] = diferencias_etapa3

# Calcular el mejor modelo por subproceso y su diferencia promedio
def obtener_mejor_modelo(series):
    conteos = series.value_counts()
    if conteos.empty:
        return None
    return conteos.idxmax()

modelo_repeticiones = final_mejores_modelos_df.apply(obtener_mejor_modelo, axis=1)
diferencia_promedio = final_diferencias_df.apply(lambda x: x.mean(), axis=1)

# Crear un DataFrame final para el mejor modelo por subproceso y su diferencia promedio
mejor_modelo_por_subproceso_df = pd.DataFrame({
    'Mejor Modelo': modelo_repeticiones,
    'Diferencia Promedio': diferencia_promedio
})

# Exportar los resultados del mejor modelo a un archivo Excel
output_file_path_final_extended = 'Resultados_Final_Modelos_Extendido.xlsx'

with pd.ExcelWriter(output_file_path_final_extended) as writer:
    final_mejores_modelos_df.to_excel(writer, sheet_name='Mejores Modelos')
    final_diferencias_df.to_excel(writer, sheet_name='Diferencias')
    mejor_modelo_por_subproceso_df.to_excel(writer, sheet_name='Mejor Modelo x Subproceso')
